In [1]:
import numpy as np
import pandas as pd
import os
import re
from tqdm import tqdm
import requests
from io import StringIO
from os import listdir
from os.path import isfile, join
import datetime as dt
from datetime import datetime, timedelta

import gridstatus

In [2]:
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f



In [3]:
FILE_PATH = "/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data"

In [4]:
ISOS = ["PJM"]

In [5]:
ISO_files = {}
ISO_data = {}
ISO_final = {}

#load in
for ISO in ISOS:
    ISO_files[ISO] = listdir_nohidden(f"{FILE_PATH}/{ISO}/old")
    ISO_data[ISO] = {}
    for file in tqdm(ISO_files[ISO]):
        compiled = re.compile(re.escape(f"{ISO}_"), re.IGNORECASE)

        ISO_data[ISO][compiled.sub("", file).replace(".csv","")] = pd.read_csv(f"{FILE_PATH}/{ISO}/old/{file}")

16it [00:05,  2.70it/s]


In [6]:
#add merge datetime
titles = {}
for key in ISO_data:
    print(key)
    titles[key]= []
    for k in ISO_data[key]:
        try:
            ISO_data[key][k]["time_tomerge"] = pd.to_datetime(ISO_data[key][k].iloc[:,0], utc=True)
        except:
            ISO_data[key][k] = ISO_data[key][k][ISO_data[key][k].index % 24 != 0]
            ISO_data[key][k]["time_tomerge"] = pd.to_datetime(ISO_data[key][k].iloc[:,0], utc=True)
            print(type(ISO_data[key][k]["time_tomerge"]))

PJM


In [7]:
ISO_final = {}
def clean_final(ISO: str, df: str):
    ISO_data[ISO][df].columns = ISO_data[ISO][df].columns.str.lower()
    no_date = [col for col in ISO_data[ISO][df].columns if "date" not in col]
    ISO_data[ISO][df] = ISO_data[ISO][df][no_date]
    try:
        ISO_data[ISO][df] = ISO_data[ISO][df].drop("unnamed: 0")
        ISO_data[ISO][df] = ISO_data[ISO][df].drop("hour")
    except:
        pass
    ISO_data[ISO][df] = ISO_data[ISO][df].sort_values(by="time_tomerge")
    ISO_data[ISO][df] = ISO_data[ISO][df].set_index("time_tomerge")
    ISO_final[ISO] = ISO_data[ISO][df]
    for column in ISO_final[ISO].columns:
        if str(ISO).lower() in column:
            ISO_final[ISO] = ISO_final[ISO].rename(columns={column: column.replace(f"{str(ISO).lower()}_","")})
    print(ISO_final[ISO])

In [8]:
for iso in ISOS:
    clean_final(iso, "final")

                           dasr_procurement_price  dasr_procurement_quantity  \
time_tomerge                                                                   
2014-01-01 05:00:00+00:00                     0.0                     5840.1   
2014-01-01 06:00:00+00:00                     0.0                     5846.7   
2014-01-01 07:00:00+00:00                     0.0                     5477.1   
2014-01-01 08:00:00+00:00                     0.0                     6297.9   
2014-01-01 09:00:00+00:00                     0.0                     5476.9   
...                                           ...                        ...   
2019-01-01 00:00:00+00:00                     0.0                     5223.8   
2019-01-01 01:00:00+00:00                     0.0                     5253.8   
2019-01-01 02:00:00+00:00                     0.0                     5223.8   
2019-01-01 03:00:00+00:00                     0.0                     5223.8   
2019-01-01 04:00:00+00:00               

In [9]:
#FOR COMBING DATA
ISO_done = ["PJM"]
FILE_PATH = "/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data/PJM/new"
for file in (listdir_nohidden(FILE_PATH)):
    if file in ["generation"]:
        dfs = []
        for f in tqdm(listdir_nohidden(f"{FILE_PATH}/{file}")):
            df = pd.read_csv(f"{FILE_PATH}/{file}/{f}")
            #df['Date'] = [f[14:24].replace("_", "-") for _ in range(len(df))]
            dfs.append(df)

        temp = pd.concat(dfs)
        os.makedirs(f"{FILE_PATH}/combined", exist_ok=True)
        temp.to_csv(f"{FILE_PATH}/combined/{file}.csv")

4it [00:00,  9.96it/s]


In [10]:
ISO_final[ISO].columns


Index(['dasr_procurement_price', 'dasr_procurement_quantity', 'dasr_cost',
       'rto_activation_cost_nonsynchr', 'mad_activation_cost_nonsynchr',
       'mad_pr_price', 'mad_sr_price', 'rto_mileage_ratio', 'rto_pr_price',
       'rto_reg_capability_price', 'rto_reg_performance_price', 'rto_sr_price',
       'mad_pr_reg', 'mad_sr_req', 'rto_pr_req', 'rto_reg_req', 'rto_sr_req',
       'mad_pr_total', 'mad_sr_total', 'rto_pr_total', 'rto_reg_total',
       'rto_sr_total', 'rto_reg_activation_quantity', 'energy_da', 'lmp_da',
       'congestion_da', 'losses_da', 'energy_rt', 'lmp_rt', 'congestion_rt',
       'losses_rt', 'exchange_rt', 'unnamed: 0', 'gas_price', 'coal', 'gas',
       'hydro', 'multiple fuels', 'nuclear', 'oil', 'other',
       'other renewables', 'solar', 'wind', 'load', 'gen_wind'],
      dtype='object')

In [11]:
ISO_done = ["PJM"]
FILE_PATH = "/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data"
new_data = {}
for ISO in ISO_done:
    sub_folder = f"{FILE_PATH}/{ISO}/new/combined"
    files = [f for f in listdir(sub_folder) if isfile(join(sub_folder, f)) and f[0]!="."]
    new_data[ISO] = {}
    print(ISO)
    for file in tqdm(files):
        new_data[ISO][file.replace(".csv", "")] = pd.read_csv(f"{sub_folder}/{file}")

PJM


100%|██████████| 11/11 [00:05<00:00,  2.01it/s]


In [12]:
print([key for key in new_data["PJM"]])

['solar', 'ancillary_dasr_procurement', 'energy_da', 'ancillary_procurement_price', 'energy_rt', 'wind', 'generation', 'ancillary_nonsynchr_cost', 'load', 'exchange', 'ancillary_procurement_quantity']


In [13]:
new_data["PJM"]["energy_rt"].columns

Index(['Unnamed: 0', 'datetime_beginning_ept', 'pnode_id', 'pnode_name',
       'voltage', 'equipment', 'type', 'zone', 'system_energy_price_rt',
       'total_lmp_rt', 'congestion_price_rt', 'marginal_loss_price_rt',
       'row_is_current', 'version_nbr'],
      dtype='object')

In [14]:
energy_rt = new_data["PJM"]["energy_rt"]
energy_rt = energy_rt[energy_rt["pnode_name"] == "PJM-RTO"]
energy_rt["time_tomerge"] = pd.to_datetime(energy_rt["datetime_beginning_ept"]) + timedelta(hours=1)
energy_rt = energy_rt[["time_tomerge", "system_energy_price_rt", "congestion_price_rt", "marginal_loss_price_rt", "total_lmp_rt"]].rename(columns={"system_energy_price_rt": "energy_rt", "congestion_price_rt": "congestion_rt", "marginal_loss_price_rt": "losses_rt", "total_lmp_rt": "lmp_rt"})
print(energy_rt)

              time_tomerge  energy_rt  congestion_rt  losses_rt     lmp_rt
0      2022-01-01 01:00:00      18.91       0.000251   0.006118  18.914703
23     2022-01-01 02:00:00      18.16       0.000000   0.005346  18.163679
46     2022-01-01 03:00:00      18.50       0.000162   0.005814  18.509309
69     2022-01-01 04:00:00      18.38       0.000711   0.009022  18.390567
92     2022-01-01 05:00:00      18.11      -0.000033   0.006453  18.119753
...                    ...        ...            ...        ...        ...
811992 2019-12-30 21:00:00      18.69       0.010342   0.008448  18.711290
812015 2019-12-30 22:00:00      18.50       0.007895   0.008187  18.516915
812038 2019-12-30 23:00:00      17.62       0.006954   0.011930  17.642217
812061 2019-12-31 00:00:00      17.47       0.009210   0.010926  17.488469
812084 2019-12-31 01:00:00      17.41       0.052813   0.009547  17.467360

[35309 rows x 5 columns]


/var/folders/cz/nv3pr_214md_31815rhfvdkh0000gn/T/ipykernel_98461/1791878446.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  energy_rt["time_tomerge"] = pd.to_datetime(energy_rt["datetime_beginning_ept"]) + timedelta(hours=1)


In [15]:
energy_da = new_data["PJM"]["energy_da"]
energy_da = energy_da[energy_da["pnode_name"] == "PJM-RTO"]
energy_da["time_tomerge"] = pd.to_datetime(energy_da["datetime_beginning_ept"]) + timedelta(hours=1)
energy_da = energy_da[["time_tomerge", "system_energy_price_da", "congestion_price_da", "marginal_loss_price_da", "total_lmp_da"]].rename(columns={"system_energy_price_da": "energy_da", "congestion_price_da": "congestion_da", "marginal_loss_price_da": "losses_da", "total_lmp_da": "lmp_da"})
print(energy_da)

              time_tomerge  energy_da  congestion_da  losses_da     lmp_da
0      2019-01-01 01:00:00      18.96      -0.017594   0.048129  18.990535
23     2019-01-01 02:00:00      18.80      -0.025540  -0.008323  18.766137
46     2019-01-01 03:00:00      18.57      -0.014702  -0.027308  18.527990
69     2019-01-01 04:00:00      18.40      -0.016699  -0.018047  18.365254
92     2019-01-01 05:00:00      18.57      -0.010931  -0.023695  18.535374
...                    ...        ...            ...        ...        ...
804264 2020-12-30 21:00:00      22.03       0.125627   0.266690  22.422317
804287 2020-12-30 22:00:00      20.53       0.058729   0.228399  20.817128
804310 2020-12-30 23:00:00      18.78       0.007678   0.117070  18.904748
804333 2020-12-31 00:00:00      18.10       0.070194   0.130718  18.300912
804356 2020-12-31 01:00:00      17.25       0.329560   0.024265  17.603824

[34973 rows x 5 columns]


/var/folders/cz/nv3pr_214md_31815rhfvdkh0000gn/T/ipykernel_98461/2463706358.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  energy_da["time_tomerge"] = pd.to_datetime(energy_da["datetime_beginning_ept"]) + timedelta(hours=1)


In [16]:
load = new_data["PJM"]["load"]
load["time_tomerge"] = pd.to_datetime(load["datetime_beginning_ept"]) + timedelta(hours=1)
load = load[["time_tomerge", "mw"]].rename(columns={"mw":"load"})
print(load)

             time_tomerge       load
0     2020-01-01 01:00:00  79182.471
1     2020-01-01 02:00:00  77573.714
2     2020-01-01 03:00:00  76202.136
3     2020-01-01 04:00:00  75674.034
4     2020-01-01 05:00:00  76238.041
...                   ...        ...
35059 2019-12-31 20:00:00  92784.789
35060 2019-12-31 21:00:00  90016.822
35061 2019-12-31 22:00:00  87332.351
35062 2019-12-31 23:00:00  84430.223
35063 2020-01-01 00:00:00  81522.416

[35064 rows x 2 columns]


In [17]:
exchange = new_data["PJM"]["exchange"]
exchange["time_tomerge"] = pd.to_datetime(exchange["datetime_ending_ept"])

exchange_average = pd.DataFrame()
exchange_average["time_tomerge"] = np.unique(pd.to_datetime(exchange["datetime_ending_ept"]))
exchange_average = exchange_average.merge(exchange[exchange["hrly_net_tie_sched"]>0].groupby(["time_tomerge"])["hrly_net_tie_sched"].sum(), on = "time_tomerge").rename(columns = {"hrly_net_tie_sched": "exchange_positive"})
exchange_average = exchange_average.merge(exchange[exchange["hrly_net_tie_sched"]<0].groupby(["time_tomerge"])["hrly_net_tie_sched"].sum(), on = "time_tomerge").rename(columns = {"hrly_net_tie_sched": "exchange_negative"})
exchange_average = exchange_average.merge(exchange.groupby(["time_tomerge"])["hrly_net_tie_sched"].sum(), on = "time_tomerge").rename(columns = {"hrly_net_tie_sched": "exchange_rt"})
exchange_average

,time_tomerge,exchange_positive,exchange_negative,exchange_rt
0,2019-01-01 01:00:00,1273.75,-4706.00,-3432.25
1,2019-01-01 02:00:00,1582.00,-5814.50,-4232.50
2,2019-01-01 03:00:00,1656.75,-5607.25,-3950.50
3,2019-01-01 04:00:00,1668.25,-5574.00,-3905.75
4,2019-01-01 05:00:00,1455.00,-5908.00,-4453.00
...,...,...,...,...
32856,2022-12-30 21:00:00,982.00,-3632.25,-2650.25
32857,2022-12-30 22:00:00,771.00,-3568.75,-2797.75
32858,2022-12-30 23:00:00,1152.50,-3669.00,-2516.50
32859,2022-12-31 00:00:00,1587.50,-3638.75,-2051.25


In [18]:
generation = new_data["PJM"]["generation"]
generation["time_tomerge"] = pd.to_datetime(generation["datetime_beginning_ept"]) + timedelta(hours=1)


In [19]:
generation_new = pd.DataFrame()
generation_new["time_tomerge"] = np.unique(pd.to_datetime(generation["datetime_beginning_ept"]) + timedelta(hours=1))
for fuel in tqdm(np.unique(generation["fuel_type"])):
    generation_new = generation_new.merge(generation[["time_tomerge","mw"]][generation["fuel_type"] == fuel], on = "time_tomerge").rename(columns={"mw": fuel})

100%|██████████| 11/11 [00:00<00:00, 22.56it/s]


In [20]:
generation_new.columns = [x.lower() for x in generation_new.columns]

generation_new = generation_new.rename(columns = {'coal':'gen_coal',
 'gas':'gen_gas',
 'hydro': 'gen_hydro',
 'multiple fuels': "gen_multiple_fuels",
 'nuclear': "gen_nuclear",
 'oil': "gen_oil",
 'other': "gen_other",
 'other renewables': "gen_other_renewables",
 'solar': "gen_solar",
 'storage': "gen_storage",
 'wind': "gen_wind"})


In [21]:
print(generation_new)

             time_tomerge  gen_coal  gen_gas  gen_hydro  gen_multiple_fuels  \
0     2019-01-01 01:00:00   17096.1  17973.6     2099.6                 3.2   
1     2019-01-01 02:00:00   17192.6  16787.6     2039.8                 3.2   
2     2019-01-01 03:00:00   17037.8  16104.1     2054.6                 3.2   
3     2019-01-01 04:00:00   16940.3  15509.3     2048.7                 3.3   
4     2019-01-01 05:00:00   16652.0  15266.8     2041.7                 3.2   
...                   ...       ...      ...        ...                 ...   
43081 2022-12-30 21:00:00   14359.3  37030.2     2172.3               313.2   
43082 2022-12-30 22:00:00   14025.6  35371.1     2064.3               314.2   
43083 2022-12-30 23:00:00   13606.7  34407.7     1784.0                 3.2   
43084 2022-12-31 00:00:00   13351.4  31816.3     1006.7                 3.1   
43085 2022-12-31 01:00:00   13130.7  29708.9      859.0                 3.2   

       gen_nuclear  gen_oil  gen_other  gen_other_r

In [25]:
ISO_final["PJM"] = ISO_final["PJM"].drop(columns=["gen_wind"])

KeyError: "['gen_wind'] not found in axis"

In [ ]:
ISO_final["PJM"].columns.sort_values()

In [ ]:
print([key for key in new_data["PJM"]])


In [26]:
prices = new_data["PJM"]["ancillary_procurement_price"]
prices["time_tomerge"] = pd.to_datetime(prices["datetime_beginning_ept"]) + timedelta(hours=1)
prices_new = pd.DataFrame()
prices_new["time_tomerge"] = np.unique(prices["time_tomerge"])
print(prices[["value", "time_tomerge"]][prices["ancillary_service"] == "RTO Mileage Ratio"])
prices_new = prices_new.merge(prices[["value", "time_tomerge"]][prices["ancillary_service"] == "RTO Mileage Ratio"]).rename(columns={"value":'rto_mileage_ratio'})
prices_new = prices_new.merge(prices[["value", "time_tomerge"]][prices["ancillary_service"] == "MAD Non-Synchronized Reserve"]).rename(columns={"value":'mad_activation_cost_nonsynchr'})
prices_new = prices_new.merge(prices[["value", "time_tomerge"]][prices["ancillary_service"] == "RTO Non-Synchronized Reserve"]).rename(columns={"value":'rto_activation_cost_nonsynchr'})

            value        time_tomerge
2        5.352964 2022-01-01 01:00:00
9        4.070268 2022-01-01 02:00:00
16       3.291446 2022-01-01 03:00:00
23       5.480822 2022-01-01 04:00:00
30       7.043057 2022-01-01 05:00:00
...           ...                 ...
252689   7.806754 2019-12-31 20:00:00
252696   6.997204 2019-12-31 21:00:00
252703   7.047283 2019-12-31 22:00:00
252710   8.758310 2019-12-31 23:00:00
252717  11.987182 2020-01-01 00:00:00

[35472 rows x 2 columns]


In [27]:
np.unique(prices["ancillary_service"])

array(['MAD Non-Synchronized Reserve', 'MAD Secondary Reserve',
       'MAD Synchronized Reserve', 'RTO Mileage Ratio',
       'RTO Non-Synchronized Reserve', 'RTO Regulation Capability',
       'RTO Regulation Performance', 'RTO Secondary Reserve',
       'RTO Synchronized Reserve'], dtype=object)

In [28]:
print(prices_new)

             time_tomerge  rto_mileage_ratio  mad_activation_cost_nonsynchr  \
0     2019-01-01 01:00:00           3.805164                            0.0   
1     2019-01-01 02:00:00           3.810625                            0.0   
2     2019-01-01 03:00:00           4.645885                            0.0   
3     2019-01-01 04:00:00           7.363433                            0.0   
4     2019-01-01 05:00:00           5.071925                            0.0   
...                   ...                ...                            ...   
37939 2022-12-31 20:00:00           6.540520                            0.0   
37940 2022-12-31 21:00:00           8.937083                            0.0   
37941 2022-12-31 22:00:00           6.242416                            0.0   
37942 2022-12-31 23:00:00           5.700740                            0.0   
37943 2023-01-01 00:00:00           9.124056                            0.0   

       rto_activation_cost_nonsynchr  
0           

In [29]:
dasr = new_data["PJM"]["ancillary_dasr_procurement"]
dasr["time_tomerge"] =  pd.to_datetime(dasr["datetime_ending_ept"]).dt.floor("H")
dasr = dasr[["time_tomerge", "dasrmcp", "total_pjm_cleared_dasr_mwh"]].rename(columns={"dasrmcp": "dasr_procurement_price", "total_pjm_cleared_dasr_mwh": "dasr_procurement_quantity"})

In [30]:
#rto_reg_capability_price =
rto = new_data["PJM"]["ancillary_procurement_quantity"][new_data["PJM"]["ancillary_procurement_quantity"]["service"] == "REG"]
rto["time_tomerge"] = pd.to_datetime(rto["datetime_beginning_ept"]).dt.floor("H")+timedelta(hours=1)

/var/folders/cz/nv3pr_214md_31815rhfvdkh0000gn/T/ipykernel_98461/2226348389.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rto["time_tomerge"] = pd.to_datetime(rto["datetime_beginning_ept"]).dt.floor("H")+timedelta(hours=1)


In [31]:
rto = rto[['time_tomerge', 'reg_ccp', 'reg_pcp', 'as_req_mw', 'total_mw']].rename(columns={'reg_ccp': 'rto_reg_capability_price', 'reg_pcp': 'rto_reg_performance_price', 'as_req_mw': 'rto_reg_req', 'total_mw': 'rto_reg_total'})

In [32]:
rto_sr = new_data["PJM"]["ancillary_procurement_quantity"][new_data["PJM"]["ancillary_procurement_quantity"]["locale"] =="PJM_RTO"]
rto_sr = rto_sr[rto_sr["service"] == "SR"]
rto_sr["time_tomerge"] = pd.to_datetime(rto_sr["datetime_beginning_ept"]).dt.floor("H")+timedelta(hours=1)
rto_sr = rto_sr[['time_tomerge', 'mcp', 'as_req_mw', 'total_mw']].rename(columns={'mcp': 'rto_sr_price', 'as_req_mw':'rto_sr_req', 'total_mw': 'rto_sr_total'})

In [33]:
mad_sr = new_data["PJM"]["ancillary_procurement_quantity"][new_data["PJM"]["ancillary_procurement_quantity"]["locale"] =="MAD"]
mad_sr = mad_sr[mad_sr["service"] == "SR"]
mad_sr["time_tomerge"] = pd.to_datetime(mad_sr["datetime_beginning_ept"]).dt.floor("H")+timedelta(hours=1)
mad_sr = mad_sr[['time_tomerge', 'mcp', 'as_req_mw', 'total_mw']].rename(columns={'mcp': 'mad_sr_price', 'as_req_mw':'mad_sr_req', 'total_mw': 'mad_sr_total'})

In [34]:
rto_pr = new_data["PJM"]["ancillary_procurement_quantity"][new_data["PJM"]["ancillary_procurement_quantity"]["locale"] =="PJM_RTO"]
rto_pr = rto_pr[rto_pr["service"] == "PR"]
rto_pr["time_tomerge"] = pd.to_datetime(rto_pr["datetime_beginning_ept"]).dt.floor("H")+timedelta(hours=1)
rto_pr = rto_pr[['time_tomerge', 'mcp', 'as_req_mw', 'total_mw']].rename(columns={'mcp': 'rto_pr_price', 'as_req_mw':'rto_pr_req', 'total_mw': 'rto_pr_total'})

In [35]:
mad_pr = new_data["PJM"]["ancillary_procurement_quantity"][new_data["PJM"]["ancillary_procurement_quantity"]["locale"] =="MAD"]
mad_pr = mad_pr[mad_pr["service"] == "PR"]
mad_pr["time_tomerge"] = pd.to_datetime(mad_pr["datetime_beginning_ept"]).dt.floor("H")+timedelta(hours=1)
mad_pr = mad_pr[['time_tomerge', 'mcp', 'as_req_mw', 'total_mw']].rename(columns={'mcp': 'mad_pr_price', 'as_req_mw':'mad_pr_req', 'total_mw': 'mad_pr_total'})

In [36]:
ISO_final["PJM"] = ISO_final["PJM"].rename(columns={'coal':'gen_coal',
 'gas':'gen_gas',
 'hydro': 'gen_hydro',
 'multiple fuels': "gen_multiple_fuels",
 'nuclear': "gen_nuclear",
 'oil': "gen_oil",
 'other': "gen_other",
 'other renewables': "gen_other_renewables",
 'solar': "gen_solar",
 'wind': "gen_wind", "mad_pr_reg": "mad_pr_req"
                                                    })

In [37]:
dfs = [energy_da, energy_rt, generation_new, load, exchange_average, rto, rto_pr, rto_sr, mad_pr, mad_sr, dasr, prices_new]
total = dfs[0]
for df in dfs[1:]:
    print(df.groupby(["time_tomerge"])[df.columns[1:]].mean())
    total = total.merge(df.groupby(["time_tomerge"])[df.columns[1:]].mean().reset_index(), how="outer")

                     energy_rt  congestion_rt  losses_rt     lmp_rt
time_tomerge                                                       
2019-01-01 01:00:00      18.03       0.000000   0.008116  18.039782
2019-01-01 02:00:00      17.77       0.000000   0.008393  17.775893
2019-01-01 03:00:00      16.36       0.000000   0.007960  16.371294
2019-01-01 04:00:00      12.27       0.000000   0.005159  12.279325
2019-01-01 05:00:00       6.24       0.000000   0.002282   6.241449
...                        ...            ...        ...        ...
2022-12-30 21:00:00      27.67       0.016360   0.013778  27.700971
2022-12-30 22:00:00      25.83       0.017902   0.012333  25.862735
2022-12-30 23:00:00      26.01       0.027443   0.009514  26.045290
2022-12-31 00:00:00      32.79       0.007247   0.014244  32.806490
2022-12-31 01:00:00      26.05       0.002246   0.010917  26.063163

[34945 rows x 4 columns]
                     gen_coal  gen_gas  gen_hydro  gen_multiple_fuels  \
time_tomerge     

In [38]:
total = total.sort_values(by="time_tomerge")


In [44]:
ISO_final["PJM"] = ISO_final["PJM"].reset_index()

In [45]:
#fix the time
ISO_final["PJM"]["time_tomerge"] = ISO_final["PJM"]["time_tomerge"]-timedelta(hours=4)

In [50]:
ISO_final["PJM"]["time_tomerge"][0]

Timestamp('2014-01-01 01:00:00+0000', tz='UTC')

In [53]:
total["time_tomerge"] = pd.to_datetime(total["time_tomerge"], utc=True)

In [54]:
combined = pd.concat([ISO_final["PJM"], total])
print(combined["time_tomerge"])

0       2014-01-01 01:00:00+00:00
1       2014-01-01 02:00:00+00:00
2       2014-01-01 03:00:00+00:00
3       2014-01-01 04:00:00+00:00
4       2014-01-01 05:00:00+00:00
                   ...           
35083   2022-12-31 20:00:00+00:00
35084   2022-12-31 21:00:00+00:00
35085   2022-12-31 22:00:00+00:00
35086   2022-12-31 23:00:00+00:00
35087   2023-01-01 00:00:00+00:00
Name: time_tomerge, Length: 78916, dtype: datetime64[ns, UTC]


In [55]:
combined.to_csv("/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/ISOs/PJM.csv")


In [ ]:
ISO_final["PJM"].columns.sort_values()